In [3]:
import re
import torch
import nltk
import numpy as np
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, BertTokenizer, BertModel, AutoModel, AutoTokenizer
from preprocess import preprocess
from skmultilearn.adapt import MLkNN, MLARAM
from skmultilearn.problem_transform import ClassifierChain
from sklearn.svm import SVC
lemmatizer = WordNetLemmatizer()
from sklearn.feature_extraction.text import TfidfVectorizer
from bert_base_cased import BertBaseCased
from bert_dataset import BertDataset
from bert_test import bert_test
from bert_train import bert_train
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, hamming_loss
from sklearn.ensemble import RandomForestRegressor

STOPWORDS = set(stopwords.words('english'))

device = 'cuda' if torch.cuda.is_available() else 'cpu'

[nltk_data] Downloading package stopwords to /home/stepan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/stepan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/stepan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /home/stepan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/stepan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/stepan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
device

'cuda'

In [5]:
df = pd.read_csv('./datasets/luxury_apparel_data/Luxury_Products_Apparel_Data.csv')

df

,Unnamed: 0,Category,SubCategory,ProductName,Description
0,8037,Accessories,Bags,"""Prada Striped Shell Belt Bag""","""One of Prada's most functional designs, this ..."
1,13670,Accessories,Socks,"""Falke - Lhasa Wool And Cashmere-blend Socks -...","""Falke - Casual yet luxurious, Falke's dark na..."
2,13983,Suits,Tuxedos,"""peak lapel tuxedo suit jacket""","""White and black linen blend peak lapel tuxedo..."
3,12081,Accessories,Gloves,"""Thom Browne Navy 4-Bar Rib Gloves""","""Pair of rib knit cashmere gloves in navy. Sig..."
4,15617,Accessories,Cufflinks,"""Alice Made This - Bayley Round Patina-brass C...","""Alice Made This - Made in the UK, these teal ..."
...,...,...,...,...,...
4995,12341,Shoes,Sneakers,"""Future Leather Low Top Sneakers""","""Leather upper. Concealed front lace-up closur..."
4996,6385,Jewelry,Watches,"""Gucci Black and Silver Striped Leather G-Time...","""Swiss-made PVD-coated stainless steel two-han..."
4997,13555,Accessories,Belts,"""two tone braces""","""Bordeaux red leather two tone braces from Fef..."
4998,10536,Activewear,Active Pants,"""Valentino - Logo Track Pants - Mens - Black""","""Valentino - Crafted in lightweight virgin-woo..."


In [27]:
df.shape

(5000, 5)

In [28]:
df['Category'].unique(), df['SubCategory'].unique()

(array(['Accessories', 'Suits', 'Jackets/Coats', 'Activewear',
        'Underwear and Nightwear', 'Shoes', 'Shirts', 'Pants', 'Jewelry',
        'Sweaters', nan], dtype=object),
 array(['Bags', 'Socks', 'Tuxedos', 'Gloves', 'Cufflinks', 'Overshirts',
        'Active Pants', 'Tech Accessories', 'Ties', 'Robes', 'Boots',
        'Coats', 'Loafers', 'Boxers & Briefs', 'Slides/Slipper', 'Eyewear',
        'Oxfords', 'Shirts', 'Pocket Squares', 'Denim', 'T-Shirts',
        'Sweatshirts/Hoodies', 'Sandals', 'Pants', 'Watches', 'Suits',
        'Derbys', 'Shorts', 'Hats', 'Sneakers', 'Pins and Clips', 'Belts',
        'Vests', 'Chinos', 'Cardigans', 'Polos', 'Jackets', nan, 'Wallets',
        'Sweaters', 'Scarves', 'Jewelry', 'Loungewear'], dtype=object))

In [29]:
df.dropna(inplace=True)
df.rename(columns={ 'Description': 'description', 'Category': 'category',  'SubCategory': 'subcategory' }, inplace=True)
df = df[['description', 'category', 'subcategory']]

In [30]:
parent_categories = sorted(df['category'].unique())

for parent_category in parent_categories:
    df[parent_category] = df['category'].apply(lambda x: 1 if parent_category in x else 0)

In [31]:
child_categories = sorted(df['subcategory'].unique())

for child_category in child_categories:
    df[child_category] = df['subcategory'].apply(lambda x: 1 if child_category in x else 0)

In [32]:
X_train, X_test = train_test_split(df, shuffle=True, test_size=0.15)

In [33]:
X_train.shape, X_test.shape

X_train = X_train.reset_index()
X_test = X_test.reset_index()

X_train.shape, X_test.shape

((4218, 51), (745, 51))

In [47]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

categories = [*parent_categories, *child_categories]

MAX_LEN = 512
BATCH_SIZE = 6
EPOCHS = 1
LEARNING_RATE = 2e-5
NUM_CLASSES = len(categories)

train_dataset = BertDataset(X_train, MAX_LEN, tokenizer, categories)
test_dataset = BertDataset(X_test, MAX_LEN, tokenizer, categories)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                          num_workers=4, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                         num_workers=4, shuffle=False, pin_memory=True)

In [ ]:
model = BertBaseCased(NUM_CLASSES)
model.load_state_dict(torch.load('Luxury_apparel_BERT.bin', map_location=torch.device(device)))
model = model.to(device)

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
optimizer = torch.optim.AdamW(params=model.parameters(),
                  lr=LEARNING_RATE, weight_decay=1e-6)

In [ ]:
# for epoch in range(EPOCHS):
#     bert_train(
#         model=model,
#         train_loader=train_loader,
#         optimizer=optimizer,
#         loss_fn=loss_fn,
#         epoch=epoch,
#         device=device,
#     )

In [ ]:
# torch.save(model.state_dict(), "./Luxury_apparel_BERT.bin")

In [ ]:
bert_test_result = bert_test(
    model=model,
    validation_loader=test_loader,
    loss_fn=loss_fn,
    device=device,
)

In [ ]:
print(f"Accuracy: {bert_test_result['accuracy']}")
print(f"Hamming loss: {np.array(bert_test_result['hamming_loss']).mean()}")
print(f"BCE loss: {np.array(bert_test_result['bce_loss']).mean()}")
print(f"Precision (macro): {np.array(bert_test_result['macro_precision']).mean()}")
print(f"Precision (micro): {np.array(bert_test_result['micro_precision']).mean()}")
print(f"Recall (macro): {np.array(bert_test_result['macro_recall']).mean()}")
print(f"Recall (micro): {np.array(bert_test_result['micro_recall']).mean()}")

### Multi-label sklearn

In [37]:
y_train = X_train[categories]

y_test = X_test[categories]

y_train.shape, y_test.shape

((4218, 52), (745, 52))

In [38]:
X_train.drop(categories, axis=1, inplace=True)
X_test.drop(categories, axis=1, inplace=True)

In [39]:
X_train['description']

0       "From the Vintage Collection Contrast stripes ...
1       "Elegance always, but above all a nod to pract...
2       "Declare your love for the French capital in M...
3       "Maison Labiche sweatshirt with \"Girls Girls ...
4       "Canali two-piece suit in plaid. Wool. Dry cle...
                              ...                        
4213    "Off-White - This black iPhone XR phone case i...
4214    "Lense: 59mm Bridge: 15mm Temple: 145mm . Tran...
4215    "1017 ALYX 9SM Nike Sponge Laser Logo Tee in W...
4216    "Multicoloured leather buckle detail slides fr...
4217    "Uniforme has added a daring oversized hem on ...
Name: description, Length: 4218, dtype: object

In [137]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(X_train['description'])

TfidfVectorizer()

In [138]:
X_train_tfidf_transformed = tfidf_vectorizer.transform(X_train['description'])
X_test_tfidf_transformed = tfidf_vectorizer.transform(X_test['description'])

In [139]:
ml_knn = MLkNN(k=10)

X_train_ml_knn = X_train_tfidf_transformed.toarray()
y_train_ml_knn = y_train.to_numpy()
X_test_ml_knn = X_test_tfidf_transformed.toarray()
y_test_ml_knn = y_test.to_numpy()

ml_knn.fit(X_train_ml_knn, y_train_ml_knn)

MLkNN()

In [ ]:
y_pred_ml_knn = ml_knn.predict(X_test_ml_knn)

In [ ]:
print(f"Accuracy: {accuracy_score(y_test_ml_knn, y_pred_ml_knn)}")
# add accuracay from lectures
print(f"Hamming loss: {hamming_loss(y_test_ml_knn, y_pred_ml_knn)}")
print(f"Precision score (macro): {precision_score(y_test_ml_knn, y_pred_ml_knn, average='macro')}")
print(f"Precision score (micro): {precision_score(y_test_ml_knn, y_pred_ml_knn, average='micro')}")
print(f"Recall score (macro): {recall_score(y_test_ml_knn, y_pred_ml_knn, average='macro')}")
print(f"Recall score (micro): {recall_score(y_test_ml_knn, y_pred_ml_knn, average='micro')}")
# Jaccard
# F1

In [ ]:
mlaram = MLARAM(threshold=0.05, vigilance=0.95)
mlaram.fit(X_train_ml_knn, y_train_ml_knn)

In [ ]:
y_pred_mlaram = mlaram.predict(X_test_ml_knn)

In [ ]:
print(f"Accuracy: {accuracy_score(y_test_ml_knn, y_pred_mlaram)}")
print(f"Hamming loss: {hamming_loss(y_test_ml_knn, y_pred_mlaram)}")
print(f"Precision score (macro): {precision_score(y_test_ml_knn, y_pred_mlaram, average='macro')}")
print(f"Precision score (micro): {precision_score(y_test_ml_knn, y_pred_mlaram, average='micro')}")
print(f"Recall score (macro): {recall_score(y_test_ml_knn, y_pred_mlaram, average='macro')}")
print(f"Recall score (micro): {recall_score(y_test_ml_knn, y_pred_mlaram, average='micro')}")

In [ ]:
cc = ClassifierChain(
    classifier = SVC(),
    require_dense = [False, True]
)

# Fails saying that the number of classes should be greater than 1
cc.fit(X_train_tfidf_transformed, y_train)

In [ ]:
y_pred_cc = cc.predict(X_test_tfidf_transformed)

In [ ]:
print(f"Accuracy: {accuracy_score(y_test, y_pred_cc)}")
print(f"Hamming loss: {hamming_loss(y_test, y_pred_cc)}")
print(f"Precision score (macro): {precision_score(y_test, y_pred_cc, average='macro')}")
print(f"Precision score (micro): {precision_score(y_test, y_pred_cc, average='micro')}")
print(f"Recall score (macro): {recall_score(y_test, y_pred_cc, average='macro')}")
print(f"Recall score (micro): {recall_score(y_test, y_pred_cc, average='micro')}")

In [ ]:
from sklearn.naive_bayes import GaussianNB
from skmultilearn.ensemble import RakelO

rakelo = RakelO(
    base_classifier=GaussianNB(),
    base_classifier_require_dense=[True, True],
    labelset_size=y_train.shape[1] // 4,
    model_count=2*len(categories)
)

rakelo.fit(X_train_ml_knn, y_train_ml_knn)

In [ ]:
y_pred_rakelo = rakelo.predict(X_test_ml_knn)

In [ ]:
print(f"Accuracy: {accuracy_score(y_test_ml_knn, y_pred_rakelo)}")
print(f"Hamming loss: {hamming_loss(y_test_ml_knn, y_pred_rakelo)}")
print(f"Precision score (macro): {precision_score(y_test_ml_knn, y_pred_rakelo, average='macro')}")
print(f"Precision score (micro): {precision_score(y_test_ml_knn, y_pred_rakelo, average='micro')}")
print(f"Recall score (macro): {recall_score(y_test_ml_knn, y_pred_rakelo, average='macro')}")
print(f"Recall score (micro): {recall_score(y_test_ml_knn, y_pred_rakelo, average='micro')}")

### DeBerta

In [11]:
import torch
from transformers import AutoModel, DebertaForSequenceClassification

debert_tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

categories = [*parent_categories, *child_categories]

MAX_LEN = 512
BATCH_SIZE = 3
EPOCHS = 1
LEARNING_RATE = 2e-5
NUM_CLASSES = len(categories)

class DeBerta(torch.nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.deberta = DebertaForSequenceClassification.from_pretrained(
            "microsoft/deberta-base",
            num_labels=n_classes,
            problem_type='multi_label_classification'
        )

    def forward(self, ids, mask, token_type_ids):
        return self.deberta(ids, attention_mask=mask,
                                token_type_ids=token_type_ids)

deberta_model = DeBerta(n_classes=NUM_CLASSES)
deberta_model = deberta_model.to(device)

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
import torch

def debert_train(model, train_loader, optimizer, loss_fn, epoch, device='cpu'):
    model.train()
    for _, data in enumerate(train_loader, 0):
        print(f"Batch: {_}")
        ids = data['ids'].to(device)
        mask = data['mask'].to(device)
        token_type_ids = data['token_type_ids'].to(device)
        targets = data['targets'].to(device)

        outputs = model(ids, mask, token_type_ids).logits

        loss = loss_fn(outputs, targets)

        if _%50 == 0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        # grad descent step
        optimizer.step()

In [13]:
class DeBertDataset(Dataset):
    def __init__(self, df, max_len, tokenizer, target_cols):
        super().__init__()
        self.df = df
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.target_cols = target_cols


    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        text = self.df['description'][index]
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            return_tensors='pt',
            return_attention_mask=True
        )

        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': ids.clone().detach().flatten(),
            'mask': mask.clone().detach().flatten(),
            'token_type_ids': token_type_ids.clone().detach().flatten(),
            'targets': torch.tensor(self.df[self.target_cols].values[index], dtype=torch.float)
        }

In [14]:
train_dataset = DeBertDataset(X_train, MAX_LEN, debert_tokenizer, categories)
test_dataset = DeBertDataset(X_test, MAX_LEN, debert_tokenizer, categories)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                          num_workers=4, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                         num_workers=4, shuffle=False, pin_memory=True)

In [15]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.AdamW(params=deberta_model.parameters(),
                  lr=LEARNING_RATE, weight_decay=1e-6)

In [16]:
# next(iter(train_loader))

In [17]:
for epoch in range(1):
    debert_train(
        model=deberta_model,
        train_loader=train_loader,
        optimizer=optimizer,
        loss_fn=loss_fn,
        epoch=epoch,
        device=device,
    )

Batch: 0
Epoch: 0, Loss:  0.6978366374969482
Batch: 1
Batch: 2
Batch: 3
Batch: 4
Batch: 5
Batch: 6
Batch: 7
Batch: 8
Batch: 9
Batch: 10
Batch: 11
Batch: 12
Batch: 13
Batch: 14
Batch: 15
Batch: 16
Batch: 17
Batch: 18
Batch: 19
Batch: 20
Batch: 21
Batch: 22
Batch: 23
Batch: 24
Batch: 25
Batch: 26
Batch: 27
Batch: 28
Batch: 29
Batch: 30
Batch: 31
Batch: 32
Batch: 33
Batch: 34
Batch: 35
Batch: 36
Batch: 37
Batch: 38
Batch: 39
Batch: 40
Batch: 41
Batch: 42
Batch: 43
Batch: 44
Batch: 45
Batch: 46
Batch: 47
Batch: 48
Batch: 49
Batch: 50
Epoch: 0, Loss:  0.32743778824806213
Batch: 51
Batch: 52
Batch: 53
Batch: 54
Batch: 55
Batch: 56
Batch: 57
Batch: 58
Batch: 59
Batch: 60
Batch: 61
Batch: 62
Batch: 63
Batch: 64
Batch: 65
Batch: 66
Batch: 67
Batch: 68
Batch: 69
Batch: 70
Batch: 71
Batch: 72
Batch: 73
Batch: 74
Batch: 75
Batch: 76
Batch: 77
Batch: 78
Batch: 79
Batch: 80
Batch: 81
Batch: 82
Batch: 83
Batch: 84
Batch: 85
Batch: 86
Batch: 87
Batch: 88
Batch: 89
Batch: 90
Batch: 91
Batch: 92
Batch: 

In [18]:
torch.save(deberta_model.state_dict(), "./Luxury_apparel_DEBERTA.bin")

In [20]:
import torch
import numpy as np
from sklearn.metrics import precision_score, recall_score, hamming_loss

def bert_test(model, validation_loader, loss_fn, device='cpu'):
    losses = []
    hl = []
    macro_precision = []
    micro_precision = []
    macro_recall = []
    micro_recall = []
    correct_predictions = 0
    num_samples = 0
    # set model to eval mode (turn off dropout, fix batch norm)
    model.eval()

    with torch.no_grad():
        for batch_idx, data in enumerate(validation_loader, 0):
            if ((batch_idx + 1) % 100) == 0:
                print(f"Batch: {batch_idx + 1}")
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids).logits

            loss = loss_fn(outputs, targets)
            losses.append(loss.item())

            # validation accuracy
            # add sigmoid
            outputs = torch.sigmoid(outputs).cpu().detach().numpy().round()
            targets = targets.cpu().detach().numpy()
            correct_predictions += np.sum(outputs==targets)
            # total number of elements in the 2D array
            num_samples += targets.size

            # Hamming loss
            hl.append(hamming_loss(targets, outputs))

            # Macro / mictor precision
            macro_precision.append(precision_score(targets, outputs, average='macro'))
            micro_precision.append(precision_score(targets, outputs, average='micro'))

            # Macro / mictor recall
            macro_recall.append(recall_score(targets, outputs, average='macro'))
            micro_recall.append(recall_score(targets, outputs, average='micro'))

    return {
        'accuracy': float(correct_predictions)/num_samples,
        'bce_loss': losses,
        'hamming_loss': hl,
        'macro_precision': macro_precision,
        'micro_precision': micro_precision,
        'macro_recall': macro_recall,
        'micro_recall': micro_recall,
    }

In [21]:
debert_test_result = bert_test(
    model=deberta_model,
    validation_loader=test_loader,
    loss_fn=loss_fn,
    device=device,
)

/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _w

Batch: 100


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _w

Batch: 200


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _w

In [23]:
print(f"Accuracy: {debert_test_result['accuracy']}")
print(f"Hamming loss: {np.array(debert_test_result['hamming_loss']).mean()}")
print(f"BCE loss: {np.array(debert_test_result['bce_loss']).mean()}")
print(f"Precision (macro): {np.array(debert_test_result['macro_precision']).mean()}")
print(f"Precision (micro): {np.array(debert_test_result['micro_precision']).mean()}")
print(f"Recall (macro): {np.array(debert_test_result['macro_recall']).mean()}")
print(f"Recall (micro): {np.array(debert_test_result['micro_recall']).mean()}")

Accuracy: 0.9855704697986577
Hamming loss: 0.014442384924312636
BCE loss: 0.0470352923145974
Precision (macro): 0.07378230872206776
Precision (micro): 0.928714859437751
Recall (macro): 0.07419421274842963
Recall (micro): 0.6824185631414547


### RandomForestRegressor

In [81]:
bert_embeds = tokenizer(X_train['description'].tolist(), truncation=True, padding='max_length', max_length=512).input_ids

In [140]:
rfc = RandomForestRegressor(max_depth=NUM_CLASSES, random_state=42)

rfc.fit(X_train_ml_knn, y_train_ml_knn)

RandomForestRegressor(max_depth=52, random_state=42)

In [147]:
# bert_embeds_test = tokenizer(X_test['description'].tolist(), truncation=True, padding='max_length', max_length=512).input_ids

y_pred_rfc = rfc.predict(X_test_ml_knn)
y_pred_rfc = np.where(y_pred_rfc >= 0.5, 1, 0)

In [148]:
print(f"Accuracy: {accuracy_score(y_test_ml_knn, y_pred_rfc)}")
print(f"Hamming loss: {hamming_loss(y_test_ml_knn, y_pred_rfc)}")
print(f"Precision score (macro): {precision_score(y_test_ml_knn, y_pred_rfc, average='macro')}")
print(f"Precision score (micro): {precision_score(y_test_ml_knn, y_pred_rfc, average='micro')}")
print(f"Recall score (macro): {recall_score(y_test_ml_knn, y_pred_rfc, average='macro')}")
print(f"Recall score (micro): {recall_score(y_test_ml_knn, y_pred_rfc, average='micro')}")

Accuracy: 0.7530201342281879
Hamming loss: 0.010144553433144037
Precision score (macro): 0.8260237006065073
Precision score (micro): 0.935708752904725
Recall score (macro): 0.661096652048082
Recall score (micro): 0.7957839262187089


/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/stepan/HSEPythonCourse/thesis/hse_thesis_final/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
